In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import matplotlib.dates as mdates

# 그래프 저장 경로 설정
save_path = 'C:/Users/PC/Desktop/caffein/montana/vehicle_counts_graphs'
save_path = save_path.replace('\\', '/')

# 날짜 리스트 생성
dates = [f'09{str(i).zfill(2)}' for i in range(1, 13)]

# 데이터가 저장된 기본 경로 설정
data_path = 'C:/Users/PC/Desktop/caffein/montana/in_out_0901_0912'
data_path = data_path.replace('\\', '/')

for date in dates:
    print(f'Processing date: {date}')
    # 파일 경로 설정
    grouped_file = os.path.join(data_path, f'{date}_merged_grouped.csv').replace('\\', '/')
    unmatched_file = os.path.join(data_path, f'{date}_merged_unmatched.csv').replace('\\', '/')
    
    # CSV 파일 로드
    grouped_df = pd.read_csv(grouped_file)
    unmatched_df = pd.read_csv(unmatched_file)
    
    # 방문 차량 수 계산
    total_vehicles = len(grouped_df) + len(unmatched_df)
    
    # 기준 날짜 설정 (예: 2023년 9월 1일)
    base_date = datetime.datetime(2023, 9, int(date[2:]))

    # 그룹된 데이터 처리
    # entry_time과 exit_time을 datetime으로 변환 (기준 날짜 포함)
    grouped_df['entry_time'] = pd.to_datetime(grouped_df['entry_time'], format='%H:%M:%S')
    grouped_df['entry_time'] = grouped_df['entry_time'].apply(lambda x: x.replace(year=base_date.year, month=base_date.month, day=base_date.day))
    
    grouped_df['exit_time'] = pd.to_datetime(grouped_df['exit_time'], format='%H:%M:%S')
    grouped_df['exit_time'] = grouped_df['exit_time'].apply(lambda x: x.replace(year=base_date.year, month=base_date.month, day=base_date.day))
    
    # 평균 체류 시간 계산 (timedelta 형식)
    m = (grouped_df['exit_time'] - grouped_df['entry_time']).mean()
    average_duration_str = str(m).split('.')[0]  # 소수점 이하 제거
    
    # Unmatched 데이터 처리
    unmatched_df['TIME'] = pd.to_datetime(unmatched_df['TIME'], format='%H:%M:%S')
    unmatched_df['TIME'] = unmatched_df['TIME'].apply(lambda x: x.replace(year=base_date.year, month=base_date.month, day=base_date.day))
    
    # 입차만 있는 경우
    unmatched_entry = unmatched_df[unmatched_df['direction'] == '입차'].copy()
    unmatched_entry['entry_time'] = unmatched_entry['TIME']
    unmatched_entry['exit_time'] = unmatched_entry['entry_time'] + m
    
    # 출차만 있는 경우
    unmatched_exit = unmatched_df[unmatched_df['direction'] == '출차'].copy()
    unmatched_exit['exit_time'] = unmatched_exit['TIME']
    unmatched_exit['entry_time'] = unmatched_exit['exit_time'] - m
    
    # Unmatched 데이터 병합
    unmatched_processed = pd.concat([unmatched_entry, unmatched_exit], ignore_index=True)
    
    # 필요한 컬럼만 선택하고 그룹된 데이터와 형식을 맞춤
    unmatched_processed = unmatched_processed[['entry_time', 'exit_time']]
    
    # 전체 데이터 병합
    all_data = pd.concat([grouped_df[['entry_time', 'exit_time']], unmatched_processed], ignore_index=True)
    
    # 시간 범위 생성 (09:00부터 23:00까지 1분 단위)
    time_range = pd.date_range(start=base_date.replace(hour=9, minute=0, second=0),
                               end=base_date.replace(hour=23, minute=0, second=0),
                               freq='1min')
    
    vehicle_counts = []

    for current_time in time_range:
        count = all_data[(all_data['entry_time'] <= current_time) & (all_data['exit_time'] >= current_time)].shape[0]
        vehicle_counts.append(count)
    
    # 그래프 생성
    plt.figure(figsize=(15, 7))
    plt.plot(time_range, vehicle_counts, marker='o')
    
    # 그래프 제목 및 설명 추가
    plt.title(f'Vehicle Count on {date}')
    plt.xlabel('Time')
    plt.ylabel('Number of Vehicles')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    
    # x축 포맷 설정
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))
    
    # 방문 차량 수와 평균 체류 시간을 그래프에 추가
    info_text = f'Total Vehicles: {total_vehicles}\nAverage Duration: {average_duration_str}'
    plt.gcf().text(0.15, 0.85, info_text, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
    
    # 그래프 저장
    graph_filename = os.path.join(save_path, f'vehicle_count_{date}.png').replace('\\', '/')
    plt.savefig(graph_filename)
    plt.close()





Processing date: 0901
Processing date: 0902
Processing date: 0903
Processing date: 0904
Processing date: 0905
Processing date: 0906
Processing date: 0907
Processing date: 0908
Processing date: 0909
Processing date: 0910
Processing date: 0911
Processing date: 0912
